In [ ]:
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

from utils.attack_algo_utils import *
from utils.graph_utils import *
from utils.utils import *

In [ ]:
# --- CHỌN PHIÊN BẢN THÍ NGHIỆM ---
experiment_id = 1
# BASE_PATH = f'graphs/{experiment_id}'
# BASE_PATH  =  f'graph_test'
BASE_PATH  =  f'graph_test_delete'


In [ ]:
# --- LOAD ENVIRONMENTAL INFORMATION ---

import torch
import networkx as nx
import numpy as np

# --- 1. Tải Môi trường (Tĩnh) ---
STATIC_FILE_PATH = f"{BASE_PATH}/graph_environment.pth"

try:
    env_data = torch.load(STATIC_FILE_PATH, weights_only=False)

    G = env_data['G_original']
    node_order = env_data['node_order']
    node_map = env_data['node_map']

    # Bạn cũng có thể lấy features gốc nếu cần
    # node_features_goc = env_data['node_features_original']

    print(f"--- Đã tải môi trường tĩnh từ '{STATIC_FILE_PATH}' ---")
    print("Tổng số node:", len(node_order))
    print("Map của 'Host 1':", node_map['Host 1'])

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy tệp '{STATIC_FILE_PATH}'.")
    print("Vui lòng kiểm tra lại experiment_id hoặc đường dẫn.")
    # Thoát hoặc xử lý lỗi nếu cần
    exit()


# --- 2. Tải Embeddings (Động) ---
NODE_EMB_PATH = f"{BASE_PATH}/node_embeddings.npy"
EDGE_EMB_PATH = f"{BASE_PATH}/edge_embeddings.npy"

try:
    nodes_emb = np.load(NODE_EMB_PATH)
    edges_emb = np.load(EDGE_EMB_PATH)

    print(f"\n--- Đã tải embedding động từ thí nghiệm {experiment_id} ---")
    print("Shape của Node Embeddings:", nodes_emb.shape)

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy tệp '{NODE_EMB_PATH}' hoặc '{EDGE_EMB_PATH}'.")
    # Thoát hoặc xử lý lỗi nếu cần
    exit()


# --- 3. Sử dụng ---
# Giờ đây bạn đã có cả hai:
# - `node_map` để biết "Host 1" là ID số mấy.
# - `nodes_emb` để lấy embedding của ID đó.

try:
    node_name = "Host 1"
    node_id = node_map[node_name]
    embedding_cua_host_1 = nodes_emb[node_id]

    print(f"\n--- Sẵn sàng cho RL ---")
    print(f"Embedding cho '{node_name}' (ID: {node_id}): \n", embedding_cua_host_1)

except KeyError:
    print(f"Lỗi: Không tìm thấy node '{node_name}' trong node_map.")
except IndexError:
    print(f"Lỗi: node_id {node_id} nằm ngoài phạm vi của 'nodes_emb' (Shape: {nodes_emb.shape})")

In [ ]:
# --- Visualize the graph ---
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
edge_labels = nx.get_edge_attributes(G, 'user')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.title("Attack Graph (Edge Weights: User Access Prob)")
plt.show()

In [ ]:
print(nodes_emb)

In [ ]:
print(edges_emb)

In [ ]:
# ======================================================================
# TẢI DỮ LIỆU MÔI TRƯỜNG TĨNH
# ======================================================================
print("--- Đang tải dữ liệu môi trường (tĩnh) ---")
# Giả định BASE_PATH đã được định nghĩa
# Giả định các class (DGI, EGraphSAGE, TGNWrapperEncoder...) đã được định nghĩa
# Giả định các thư viện (torch, F, dgl) đã được import

env_data = torch.load(f"{BASE_PATH}/graph_environment.pth",weights_only=False)

g_dgl = env_data.get('g_dgl')
if g_dgl is None:
    G_original = env_data['G_original']
    g_dgl = dgl.from_networkx(G_original, node_attrs=['state', 'priority'], edge_attrs=['user', 'root'])
    g_dgl.ndata['h'] = env_data['node_features_original']
    g_dgl.edata['h'] = env_data['edge_features_original']
    print("Đã tạo lại g_dgl từ G_original.")

original_edge_features = env_data['edge_features_original']
original_node_features = env_data['node_features_original']
static_priority_features = original_node_features[:, 1].unsqueeze(1) # Cột priority

print("[THÀNH CÔNG] Đã tải xong dữ liệu môi trường.")

# ======================================================================
# TẢI MODEL GNN ĐÃ HUẤN LUYỆN (ĐÃ SỬA LỖI)
# ======================================================================
print("--- Đang tải cấu hình và trọng số GNN ---")

MODEL_STATE_PATH = f"{BASE_PATH}/dgi_model_state_dict.pth"
CONFIG_FILE_PATH = f"{BASE_PATH}/model_config.pth"

try:
    # --- 3.1: Tải file cấu hình ---
    config = torch.load(CONFIG_FILE_PATH, weights_only=False)
    print(f"Đã tải cấu hình: {config}")

    # --- 3.2: Khởi tạo mô hình rỗng (ĐÃ SỬA) ---
    
    # Lấy số nút gốc từ đồ thị đã tải
    NUM_NODES_ORIGINAL = g_dgl.num_nodes() # Đây sẽ là 9
    
    # Giả định các tham số TGN này khớp với lúc huấn luyện
    # (Vì chúng không có trong file config)
    MEMORY_DIM = 32 
    MSG_DIM = 32
    
    print(f"Khởi tạo TGNWrapperEncoder với {NUM_NODES_ORIGINAL} nút (gốc)...")
    
    # Khởi tạo đúng TGNWrapperEncoder
    encoder = TGNWrapperEncoder(
        num_nodes=NUM_NODES_ORIGINAL, # 9 nút
        node_feat_dim=config['NDIM_IN'],
        edge_feat_dim=config['EDIM'],
        memory_dim=MEMORY_DIM,
        msg_dim=MSG_DIM,
        sage_n_hidden=config['N_HIDDEN'],
        sage_n_out=config['N_OUT'],
        sage_n_layers=config['N_LAYERS'],
        sage_activation=F.leaky_relu
    )

    # Khởi tạo DGI với TGNWrapperEncoder
    dgi_model_to_load = DGI(encoder) # Model này có memory [9, 32]
# --- 3.3: Tải trọng số đã lưu (SỬA CƠ CHẾ TẢI) ---
    print("Đang tải state_dict (với cơ chế xử lý mismatch)...")
    
    # Tải checkpoint (chùm chìa khóa) vào một biến riêng
    checkpoint_state = torch.load(MODEL_STATE_PATH, weights_only=False)
    
    # Tách 'encoder.memory' ra khỏi checkpoint
    memory_from_checkpoint = checkpoint_state.pop("encoder.memory", None)
    
    if memory_from_checkpoint is None:
        raise KeyError("Không tìm thấy 'encoder.memory' trong checkpoint. File model có thể bị hỏng.")
        
    # Lấy số nút từ checkpoint (ví dụ: 8)
    num_nodes_checkpoint = memory_from_checkpoint.shape[0]
    
    # BƯỚC 1: Luôn tải tất cả các trọng số KHÁC (GRU, Linear, Sage...)
    # Dùng strict=False vì checkpoint_state BỊ THIẾU 'encoder.memory'
    # và cũng có thể bị thiếu các hàm 'delete' (edge_del_msg_fn...)
    # mà model MỚI (dgi_model_to_load) đang có.
    print("Đang tải các trọng số chung (GRU, SAGE, v.v.)...")
    dgi_model_to_load.load_state_dict(checkpoint_state, strict=False)
    print("Đã tải các trọng số chung thành công.")

    # BƯỚC 2: Xử lý sao chép memory thủ công
    
    # Lấy số nút của model HIỆN TẠI (ví dụ: 9)
    num_nodes_model = dgi_model_to_load.encoder.num_nodes
    
    # Xác định số lượng nút tối thiểu để sao chép
    num_to_copy = min(num_nodes_checkpoint, num_nodes_model)
    
    print(f"Đang sao chép memory: Checkpoint ({num_nodes_checkpoint} nút) -> Model ({num_nodes_model} nút).")
    
    with torch.no_grad():
        # Lấy tensor memory của model hiện tại
        current_model_memory = dgi_model_to_load.encoder.memory.data
        
        # Lấy phần memory tương ứng từ checkpoint
        checkpoint_memory_slice = memory_from_checkpoint[:num_to_copy]
        
        # Ghi đè phần chung
        # - Nếu model=9, checkpoint=8: Ghi đè 8 nút đầu tiên của model
        # - Nếu model=7, checkpoint=8: Ghi đè 7 nút đầu tiên của model
        current_model_memory[:num_to_copy] = checkpoint_memory_slice
    
    print(f"Đã sao chép thủ công memory của {num_to_copy} nút chung.")
    
    # In thông báo trạng thái
    if num_nodes_model > num_nodes_checkpoint:
        # Ví dụ: Model=9, Checkpoint=8
        print(f"  -> Model có {num_nodes_model - num_nodes_checkpoint} nút mới. Các nút này sẽ giữ memory khởi tạo (zeros).")
    elif num_nodes_checkpoint > num_nodes_model:
        # Ví dụ: Model=7, Checkpoint=8
        print(f"  -> Model nhỏ hơn checkpoint. Đã bỏ qua {num_nodes_checkpoint - num_nodes_model} nút từ checkpoint.")
    else:
        # Ví dụ: Model=8, Checkpoint=8
        print("  -> Kích thước memory khớp hoàn toàn.")

    print("Tải state_dict thành công.")
    
except Exception as e:
    print(f"\n[LỖI] Có lỗi xảy ra khi tải model: {e}")
    trained_encoder = None



In [ ]:
# def train_dqn(env, num_episodes, batch_size=10, gamma=0.99, epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.995):
#     global best_checkpoint, best_episode
#     state_size = env.num_nodes
#     action_space_size = env.get_action_space_size()
#     print('state_size', state_size)
#     print('action_space_size', action_space_size)
#     # Initialize DQN and target network
#     policy_net = DQN(state_size, action_space_size)
#     target_net = DQN(state_size, action_space_size)
#     target_net.load_state_dict(policy_net.state_dict())
#     target_net.eval()
#
#     optimizer = optim.Adam(policy_net.parameters(), lr=0.001)
#     replay_buffer = ReplayBuffer(capacity=10000)
#     epsilon = epsilon_start
#     total_reward = 0
#     dsp = 0
#     best_dsp = 0
#     interval_check = num_episodes // 10  # Mỗi num_episodes/10
#     interval_save = num_episodes // 5   # Lưu sau mỗi num_episodes/5
#
#     for episode in range(1, num_episodes+1):
#         state = env.reset()
#         done = False
#
#         exploration_counter = defaultdict(int)
#
#         while not done:
#             if random.random() < epsilon:
#                 # Chọn ngẫu nhiên index hợp lệ
#                 action_idx = sample_valid_index(action_space_size, env.num_honeypot_nodes, exploration_counter)
#             else:
#                 with torch.no_grad():
#                     state_tensor = torch.FloatTensor(state).unsqueeze(0)
#                     q_values = policy_net(state_tensor).squeeze(0)  # shape: [action_space_size]
#
#                     # Lọc q_values chỉ lấy index hợp lệ
#                     valid_indices = [idx for idx in range(action_space_size) if is_valid_index(idx, env.num_honeypot_nodes)]
#                     valid_q_values = q_values[valid_indices]
#                     # Lấy chỉ số trong valid_indices có q_value max
#                     max_idx_in_valid = torch.argmax(valid_q_values).item()
#                     # Map về action_idx thực
#                     action_idx = valid_indices[max_idx_in_valid]
#
#             action = index_to_action(action_idx, env.num_honeypot_nodes)
#             next_state, reward, done, path, captured = env.step(action)
#             action_idx = action_to_index(action, env.num_honeypot_nodes)
#
#             # Store experience
#             replay_buffer.push(state, action_idx, reward, next_state, done)
#             state = next_state
#             total_reward += reward
#             if reward == 1:
#                 dsp += 1
#             # Train if enough experiences
#             if len(replay_buffer) >= batch_size:
#                 states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)
#
#                 states = torch.FloatTensor(states)
#                 actions = torch.LongTensor(actions)
#                 rewards = torch.FloatTensor(rewards)
#                 next_states = torch.FloatTensor(next_states)
#                 dones = torch.FloatTensor(dones)
#
#                 # Compute Q-values
#                 q_values = policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)
#
#                 # Compute target Q-values
#                 with torch.no_grad():
#                     next_q_values = target_net(next_states).max(1)[0]
#                     targets = rewards + (1 - dones) * gamma * next_q_values
#
#                 # Compute loss
#                 loss = nn.MSELoss()(q_values, targets)
#
#                 # Optimize
#                 optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()
#
#         # Update target network
#         if episode % 10 == 0:
#             target_net.load_state_dict(policy_net.state_dict())
#
#         # Decay epsilon
#         epsilon = max(epsilon_end, epsilon * epsilon_decay)
#
#         # Logging
#         if episode % interval_check == 0:
#             placement = []
#             for i in range(2):  # Two honeypots
#                 node_idx = np.argmax(action[i])
#                 node_name = env.honeypot_nodes[node_idx]
#                 placement.append(f"Honeypot {i} -> {node_name}\n")
#             print(f"Episode {episode}, Total Reward: {total_reward}, Epsilon: {epsilon:.3f}, Defense Success Probability: {dsp/interval_check}%\n")
#             print("".join(placement))
#             print(path)
#             total_reward = 0
#
#             # Log ra DSP lớn nhất sau mỗi num_episodes/10 iterations
#             if dsp > best_dsp:
#                 best_dsp = dsp
#                 best_episode = episode
#                 best_checkpoint = {
#                     'policy_net_state_dict': deepcopy(policy_net.state_dict()),
#                     'target_net_state_dict': deepcopy(target_net.state_dict()),
#                     'optimizer_state_dict': deepcopy(optimizer.state_dict()),
#                 }
#             # Reset DSP
#             dsp = 0
#
#
#         # Save ra DSP lớn nhất sau mỗi num_episodes/5 iterations
#         if (episode + 1) % interval_save == 0 and best_checkpoint is not None:
#             path = f'./Saved_Model/dqn_model.pth'
#             torch.save({
#                 'policy_net_state_dict': best_checkpoint['policy_net_state_dict'],
#                 'target_net_state_dict': best_checkpoint['target_net_state_dict'],
#                 'optimizer_state_dict': best_checkpoint['optimizer_state_dict'],
#                 'episode': best_episode},
#                 path)
#             print(f'Saved model with best DSP {best_dsp} at episode {best_episode} to {path}')
#
#             best_dsp = 0
#             best_episode = 0
#             best_checkpoint = None
#
#     return policy_net

In [ ]:
import subprocess
def train_dqn(env, num_episodes, batch_size=10, gamma=0.99, epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.995):
    global best_checkpoint, best_episode
    # 1. Reset env để lấy state (embedding) ban đầu
    state = env.reset() # state giờ là Tensor [num_nodes, embedding_dim]

    # 2. Tính toán state_size đã làm phẳng
    num_nodes = state.shape[0]
    embedding_dim = state.shape[1]
    state_size = num_nodes * embedding_dim  # <--- Kích thước input mới cho DQN

    action_space_size = env.get_action_space_size()

    print('state_size (flattened):', state_size) # <--- Cập nhật log
    print('action_space_size', action_space_size)

    # 3. Khởi tạo DQN với state_size mới
    policy_net = DQN(state_size, action_space_size)
    target_net = DQN(state_size, action_space_size)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()

    optimizer = optim.Adam(policy_net.parameters(), lr=0.001)
    replay_buffer = ReplayBuffer(capacity=10000)
    epsilon = epsilon_start
    total_reward = 0
    dsp = 0
    best_dsp = 0
    interval_check = num_episodes // 10
    interval_save = num_episodes // 5
    
    for episode in range(1, num_episodes + 1):
        # Reset state cho các episode sau
        if episode > 1:
            state = env.reset() # <--- state là Tensor embedding

        done = False
        exploration_counter = defaultdict(int)

        while not done:
            if random.random() < epsilon:
                action_idx = sample_valid_index(action_space_size, env.num_honeypot_nodes, exploration_counter)
            else:
                with torch.no_grad():
                    # --- THAY ĐỔI: Flatten state tensor ---
                    # Chuyển [N, D] -> [1, N*D]
                    state_tensor = state.flatten().unsqueeze(0)
                    q_values = policy_net(state_tensor).squeeze(0)

                    # (Logic lọc q_values giữ nguyên)
                    valid_indices = [idx for idx in range(action_space_size) if is_valid_index(idx, env.num_honeypot_nodes)]
                    valid_q_values = q_values[valid_indices]
                    max_idx_in_valid = torch.argmax(valid_q_values).item()
                    action_idx = valid_indices[max_idx_in_valid]

            action = index_to_action(action_idx, env.num_honeypot_nodes)

            # --- next_state giờ cũng là Tensor embedding ---
            next_state, reward, done, path, captured = env.step(action)
            action_idx = action_to_index(action, env.num_honeypot_nodes)

            # Store experience (state và next_state là Tensors)
            replay_buffer.push(state, action_idx, reward, next_state, done)
            state = next_state
            total_reward += reward
            if reward == 1:
                dsp += 1

            # Train if enough experiences
            if len(replay_buffer) >= batch_size:
                # --- THAY ĐỔI: Replay buffer giờ trả về Tensors ---
                states_batch, actions_batch, rewards_batch, next_states_batch, dones_batch = replay_buffer.sample(batch_size)

                # states_batch là [B, N, D], actions_batch là [B], rewards_batch là [B, 1], ...

                # --- THAY ĐỔI: Flatten state batches ---
                # Chuyển [B, N, D] -> [B, N*D]
                states_flat = states_batch.flatten(start_dim=1)
                next_states_flat = next_states_batch.flatten(start_dim=1)

                # Compute Q-values
                q_values_all = policy_net(states_flat)
                # Dùng actions_batch để lấy Q-value của action đã chọn
                q_values = q_values_all.gather(1, actions_batch.long().unsqueeze(1)).squeeze(1)

                # Compute target Q-values
                with torch.no_grad():
                    # Dùng next_states_flat
                    next_q_values = target_net(next_states_flat).max(1)[0]
                    # Squeeze rewards và dones để khớp kích thước [B]
                    targets = rewards_batch.squeeze(1) + (1 - dones_batch.squeeze(1)) * gamma * next_q_values

                # Compute loss
                loss = nn.MSELoss()(q_values, targets)

                # Optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        # Update target network
        if episode % 10 == 0:
            target_net.load_state_dict(policy_net.state_dict())

        # Decay epsilon
        epsilon = max(epsilon_end, epsilon * epsilon_decay)

        # Logging (Giữ nguyên)
        if episode % interval_check == 0:
            placement = []
            for i in range(2):
                node_idx = np.argmax(action[i])
                node_name = env.honeypot_nodes[node_idx]
                placement.append(f"Honeypot {i} -> {node_name}\n")
            print(f"Episode {episode}, Total Reward: {total_reward}, Epsilon: {epsilon:.3f}, Defense Success Probability: {dsp/interval_check}%\n")
            print("".join(placement))
            print(path)
            total_reward = 0

            if dsp > best_dsp:
                best_dsp = dsp
                best_episode = episode
                best_checkpoint = {
                    'policy_net_state_dict': deepcopy(policy_net.state_dict()),
                    'target_net_state_dict': deepcopy(target_net.state_dict()),
                    'optimizer_state_dict': deepcopy(optimizer.state_dict()),
                }
            dsp = 0
        
        # Save (Giữ nguyên)
        if (episode + 1) % interval_save == 0 and best_checkpoint is not None:
            path = f'./saved_model/dqn_model.pth'
            torch.save({
                'policy_net_state_dict': best_checkpoint['policy_net_state_dict'],
                'target_net_state_dict': best_checkpoint['target_net_state_dict'],
                'optimizer_state_dict': best_checkpoint['optimizer_state_dict'],
                'episode': best_episode},
                path)
            print(f'Saved model with best DSP {best_dsp} at episode {best_episode} to {path}')

            best_dsp = 0
            best_episode = 0
            best_checkpoint = None

    return policy_net

In [ ]:
plt.figure(figsize=(30, 36))
pos = nx.spring_layout(G)

nx.draw_networkx_nodes(G, pos, node_color='orange', node_size=2000)
nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')

nx.draw_networkx_edges(
    G, pos,
    edge_color='gray',
    arrows=True,
    arrowstyle='->',
    arrowsize=50,
    connectionstyle='arc3,rad=0.2'
)

# Vẽ nhãn trên cạnh
edge_labels = {(u, v): f"user={d['user']}, root={d['root']}" for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=12)

plt.axis('off')
plt.show()


In [ ]:
# Initialize environment and train
algo = global_weighted_random_attack
# algo = greedy_attack_priority_queue
# Tạo một bản sao của đồ thị cho môi trường
G_new_env = deepcopy(G)

env = NetworkEnv(
    G_new=G_new_env,
    attack_fn=algo,
    g_dgl=g_dgl,
    encoder=encoder,
    original_node_features=original_node_features,
    original_edge_features=original_edge_features,
    node_map=node_map,
    goal="Data Server"  # (Hoặc goal bạn muốn)
)
!mkdir saved_model
# --- 3. HUẤN LUYỆN (Như cũ) ---
num_episode = 2000
model = train_dqn(env, num_episode)

In [ ]:
evaluate_model(model,env)   